# Initializing Packages and Libraries

In [39]:
# Installing Packages
!pip install alpaca-py
!pip install alpaca-trade-api
!pip install selenium
!pip install webdriver-manager
!pip install beautifulsoup4
!pip install pandas
!pip install python-dateutil
!pip install selenium
!pip install beautifulsoup4

# Importing Libraries
import alpaca_trade_api as alpaca
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# API Keys

In [ ]:
# API key and secret key see https://app.alpaca.markets/paper/dashboard/overview to generate (API keys)
API_KEY = '' # Add here your API key
API_SECRET = '' # Add here your API secret key

api = alpaca.REST(API_KEY, API_SECRET, 'https://paper-api.alpaca.markets')
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA3R6HV88HU3',
    'accrued_fees': '0',
    'admin_configurations': {},
    'balance_asof': '2025-04-10',
    'bod_dtbp': '0',
    'buying_power': '49192.59',
    'cash': '156961.49',
    'created_at': '2025-01-09T07:12:10.708072Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 1,
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '0',
    'effective_buying_power': '49192.59',
    'equity': '95457.51',
    'id': '4d1b44aa-a5e9-4a26-8e65-643967f94440',
    'initial_margin': '70714.44',
    'intraday_adjustments': '0',
    'last_equity': '92556.4962622587',
    'last_maintenance_margin': '47160.49',
    'long_market_value': '35745.38',
    'maintenance_margin': '44378.16',
    'multiplier': '2',
    'non_marginable_buying_power': '24743.07',
    'options_approved_level': 3,
    'options_buying_power': '42905.6',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_reg_taf

# IEX Trickers

## Scrapping Tickers

In [41]:
# Alpaca offers data from IEX exchange (https://www.iexexchange.io/)
# Tickers can be scrapped from the IEX Trading website (https://iextrading.com/trading/eligible-symbols/)
def get_tickers(url):
    # Set up Selenium Chrome options for headless mode.
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # 🛠️ Use ChromeDriverManager for auto-install
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # Load the page
    driver.get(url)

    # Wait for a few seconds to allow the page and JavaScript to load completely.
    time.sleep(5)

    # Get the page source and then quit the driver
    html = driver.page_source
    driver.quit()

    # Parse the page source using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the table with eligible tickers.
    # Adjust the selector if necessary based on the actual HTML structure.
    table = soup.find('table')
    if not table:
        print("Could not locate the table containing the tickers.")
        return []

    tickers = []
    # Loop through each row in the table.
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if cells:
            # Assume the first cell holds the ticker symbol.
            ticker = cells[0].get_text(strip=True)
            tickers.append(ticker)

    return tickers

if __name__ == "__main__":
    url = "https://iextrading.com/trading/eligible-symbols/"
    tickers = get_tickers(url)
    if tickers:
        print("Retrieved tickers:")
        print(tickers)
    else:
        print("No tickers found.")


No tickers found.


## Cleaning Ticker Symbols

In [42]:
def remove_symbols_with_special_characters(symbols):
    valid_symbols = [symbol for symbol in symbols if re.match("^[a-zA-Z0-9]*$", symbol)]
    return valid_symbols

str_converter = []
for ticker in tickers:
    str_converter.append(str(ticker))
tickers = remove_symbols_with_special_characters(str_converter)
tickers

[]

## Stock Data for Tickers

In [43]:
# Initializing client for historical stock data
stock_client = StockHistoricalDataClient(API_KEY,  API_SECRET)

df_past = {}
start = datetime.now() - relativedelta(days=2)
end = datetime.now() - relativedelta(days=1)
print(start)
print(end)

batch_size = max(1, len(tickers) // 100)
df_past = pd.DataFrame()

limit_date = datetime.now() - relativedelta(years=10)

while df_past.empty and start > limit_date:
    for i in range(0, len(tickers), batch_size):
        tickers_batch = tickers[i:i + batch_size]

        request_params = StockBarsRequest(
            symbol_or_symbols=tickers_batch,
            timeframe=TimeFrame.Day,
            start=start,
            end=end,
            feed="iex"  # optional: ensures compatibility with free tier
        )

        bars = stock_client.get_stock_bars(request_params)

        if not bars.df.empty:
            df_past = pd.concat([df_past, bars.df])

    start -= timedelta(days=1)
    end -= timedelta(days=1)

if df_past.empty:
    print("No data found in the last 10 years.")
else:
    print("Data loaded. Saving to CSV...")
    df_past.to_csv("stock_data.csv", index=False)
    print("Saved as stock_data.csv")

2025-04-09 10:25:11.692151
2025-04-10 10:25:11.692219
No data found in the last 10 years.
